In [42]:
#########
# START: COPIED FROM <https://github.com/PlusLabNLP/Sonnet-Gen>
# This code was HEAVILY REFACTORED. A lot of the code is my own. However, I borrowed the code template from github.
##########

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.6/104.6 kB 3.2 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.18
    Uninstalling urllib3-1.26.18:
      Successfully uninstalled urllib3-1.26.18
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
botocore 1.33.11 requires urllib3<2.1,>=1.25.4; python_version >= "3.10", but you have urllib3 2.1.0 which is incompatible.
flair 0.13.0 requires urllib3<2.0.0,>=1.0.0, but you have urllib3 2.1.0 which is incompatible.


In [43]:
# import libraries
import numpy as np
import pandas as pd
from keybert import KeyBERT
import torch
import yake
import json
import re

import nltk
from nltk import pos_tag, word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
import spacy
nlp = spacy.load('en_core_web_sm')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [44]:
# We need 3 key words per line of poem
# So ngram size is 1 (keyphrase_ngram_range=(1, 1))
# So we dont want sop words

algorithm = KeyBERT()

In [45]:
# Read in data
file_1 ='ready_poems.csv'
file_2 ='news_summaries.csv'
file_3 ='roc.csv'

df_1 = pd.read_csv(file_1)
df_2 = pd.read_csv(file_2)
df_3 = pd.read_csv(file_3)

In [46]:
# Preprocess ROC data - OWN CODE
# Find the columns with "sentence" in the title
sentence_columns = [col for col in df_3.columns if 'sentence' in col.lower()]

# Combine the selected columns into a new column
df_3['text'] = df_3[sentence_columns].apply(lambda row: ' '.join(row), axis=1)

In [47]:
df_1.rename(columns={'content': 'text'}, inplace=True)
df_2.rename(columns={'headlines': 'title'}, inplace=True)
df_3.rename(columns={'storytitle': 'title'}, inplace=True)

In [48]:
df_1 = df_1[['title', 'text']]
df_1.head()

,title,text
0,Do not go gentle into that good night,do not go gentle into that good night NEWLINE ...
1,How Do I Love Thee? (Sonnet 43),how do i love thee let me count the ways NEWLI...
2,Shall I compare thee to a summer’s day? (Sonne...,shall i compare thee to a summers day NEWLINE ...
3,If—,if you can keep your head when all about you N...
4,Nothing Gold Can Stay,natures first green is gold NEWLINE her hardes...


In [49]:
df_2 = df_2[['title', 'text']]
df_2.head()

,title,text
0,"CBI books Rajasthan CM's brother, 14 others in...",The CBI has registered a case against Rajastha...
1,Fan shares selfie taken by SRK with his father...,Twitter user Rohan Mukherjee shared that his f...
2,European Commission recommends Ukraine be gran...,The European Commission on Friday recommended ...
3,Jos Buttler smashes England's second-fastest O...,England's Jos Buttler on Friday smashed the se...
4,Paytm CEO Vijay Shekhar buys 1.7 lakh shares o...,Paytm Founder and CEO Vijay Shekhar Sharma has...


In [50]:
df_3 = df_3[['title', 'text']]
df_3.head()

,title,text
0,David Drops the Weight,David noticed he had put on a lot of weight re...
1,Frustration,Tom had a very short temper. One day a guest m...
2,Marcus Buys Khakis,Marcus needed clothing for a business casual e...
3,Different Opinions,Bobby thought Bill should buy a trailer and ha...
4,Overcoming shortcomings,John was a pastor with a very bad memory. He t...


**Generating Keywords for Dataset 1: Poetry**

In [51]:
titles = df_1['title']
texts = df_1['text']

In [52]:
# Checks for proper noun using POS Tags. OWN CODE
def has_proper_noun(text):
    # Tokenize the text and get part-of-speech tags
    words = word_tokenize(text)
    pos_tags = pos_tag(words)

    # Check if any word is tagged as a proper noun (NNP or NNPS)
    return any(pos == 'NNP' or pos == 'NNPS' for _, pos in pos_tags)

In [55]:
# Refactored to include my own preprocessing. 
tokens = []
count = 0
text_data = []
title_data = []
sentences = []

for sen,tit in zip(texts, titles):
    tit = re.sub(r'[^a-zA-Z\s]', '', tit)
    tit = tit.lower()
    tokenized = word_tokenize(tit)
    pos = has_proper_noun(tit)

    if len(tokenized) <= 2 or pos:
        continue
    try:

        sen = re.sub(r'[^a-zA-Z\s.]', '', sen)
        sen = sen.replace(' NEWLINE', '.')
        token_len = len(sen.split())
        sen_len = len(sen.split('.'))

        if 5<=sen_len<=50:
            count += 1
            tokens.append(token_len)
            text_data.append(sen)
            title_data.append(tit)
            sentences.append(sen_len)
    except:
        continue


print(count)
print('average tokens of story:', np.mean(tokens))
print('average sentences of story:', np.mean(sentences))
print('average tokens per sentence:', np.mean(tokens)/np.mean(sentences))

11661
average tokens of story: 161.81356658948633
average sentences of story: 23.581939799331103
average tokens per sentence: 6.861758106687905


In [56]:
len(title_data), len(text_data)

(11661, 11661)

In [57]:
title_data = title_data[:7000]
text_data = text_data[:7000]

In [58]:
len(text_data), len(title_data)

(7000, 7000)

In [61]:
# Simplified format of output to exclude irrelevant fields
poems = []
for sen, tit in zip(text_data, title_data):

    data = {}
    data['Theme'] = tit
    sents = sen.split('.')
    final_keywords = []
    for sent in sents:
        yake = algorithm.extract_keywords(sent, keyphrase_ngram_range=(1, 1), stop_words=None)
        keywords = [kw[0] for kw in yake]
        if len(keywords) >= 2:
            final_keywords.append(keywords)

    data['keywords'] = final_keywords
    poems.append(data)
    if len(poems) % 500 == 1:

        json.dump(poems, open('poetry_keybert_keywords''.json','w'))

KeyboardInterrupt: ignored

**Generating Keywords for Dataset 2: News**

In [ ]:
titles = df_2['title']
texts = df_2['text']

In [ ]:
# Checks for proper noun in text using POS tagging. OWN CODE
def has_proper_noun(text):
    # Tokenize text and get pos tags
    words = word_tokenize(text)
    pos_tags = pos_tag(words)

    # Return True if any word is tagged as a proper noun
    return any(pos == 'NNP' or pos == 'NNPS' for _, pos in pos_tags)

In [ ]:
# REFACTORED GITHUB CODE
tokens = []
count = 0
text_data = []
title_data = []
sentences = []

for sen,tit in zip(texts, titles):
    tit = re.sub(r'[^a-zA-Z\s]', '', tit)
    tit = tit.lower()
    tokenized = word_tokenize(tit)
    pos = has_proper_noun(tit)

    if len(tokenized) <= 2 or pos:
        continue
    try:

        sen = re.sub(r'[^a-zA-Z\s.]', '', sen)
        sen = sen.lower()
        sen = sen.replace(' NEWLINE', '.')
        token_len = len(sen.split())
        sen_len = len(sen.split('.'))

        if 5<=sen_len<=50:
            count += 1
            tokens.append(token_len)
            text_data.append(sen)
            title_data.append(tit)
            sentences.append(sen_len)
    except:
        continue


print(count)
print('average tokens of story:', np.mean(tokens))
print('average sentences of story:', np.mean(sentences))
print('average tokens per sentence:', np.mean(tokens)/np.mean(sentences))

In [ ]:
len(title_data), len(text_data)

In [ ]:
title_data = title_data[:5000]
text_data = text_data[:5000]

In [ ]:
len(text_data), len(title_data)

In [ ]:
news = []
for sen, tit in zip(text_data, title_data):

    data = {}
    data['Theme'] = tit
    sents = sen.split('.')
    final_keywords = []
    for sent in sents:
        yake = algorithm.extract_keywords(sent, keyphrase_ngram_range=(1, 1), stop_words=None)
        keywords = [kw[0] for kw in yake]
        if len(keywords) >= 2:
            final_keywords.append(keywords)

    data['keywords'] = final_keywords
    news.append(data)

    json.dump(news, open('news_keybert_keywords''.json','w'))

**Generating Keywords for Dataset 3: ROC**

In [ ]:
titles = df_3['title']
texts = df_3['text']

In [ ]:
# Refactored GITHUB Code
tokens = []
count = 0
text_data = []
title_data = []
sentences = []

for sen,tit in zip(texts, titles):
    tit = re.sub(r'[^a-zA-Z\s]', '', tit)
    tit = tit.lower()
    tokenized = word_tokenize(tit)
    pos = has_proper_noun(tit)

    if len(tokenized) <= 2 or pos:
        continue
    try:

        sen = re.sub(r'[^a-zA-Z\s.]', '', sen)
        sen = sen.lower()
        sen = sen.replace(' NEWLINE', '.')
        token_len = len(sen.split())
        sen_len = len(sen.split('.'))

        if 5<=sen_len<=50:
            count += 1
            tokens.append(token_len)
            text_data.append(sen)
            title_data.append(tit)
            sentences.append(sen_len)
    except:
        continue


print(count)
print('average tokens of story:', np.mean(tokens))
print('average sentences of story:', np.mean(sentences))
print('average tokens per sentence:', np.mean(tokens)/np.mean(sentences))

In [ ]:
len(title_data), len(text_data)

In [ ]:
title_data = title_data[:7000]
text_data = text_data[:7000]

In [ ]:
len(text_data), len(title_data)

In [ ]:
roc = []
for sen, tit in zip(text_data, title_data):

    data = {}
    data['Theme'] = tit
    sents = sen.split('.')
    final_keywords = []
    for sent in sents:
        yake = algorithm.extract_keywords(sent, keyphrase_ngram_range=(1, 1), stop_words=None)
        keywords = [kw[0] for kw in yake]
        if len(keywords) >= 2:
            final_keywords.append(keywords)

    data['keywords'] = final_keywords
    roc.append(data)

    json.dump(roc, open('roc_keybert_keywords''.json','w'))

In [ ]:
#########
# END: COPIED FROM <https://github.com/PlusLabNLP/Sonnet-Gen>
# This code was HEAVILY REFACTORED. A lot of the code is my own. However, I borrowed the code template from github.
##########